In [1]:
from time import time
from matplotlib import image
from matplotlib import pyplot as plt
import json
from tqdm import tqdm
import pandas as pd
import os
import json
from ipywidgets import *
from IPython.display import display, HTML
import re

In [2]:
data_path = './data/2016_playoffs.json'
base_image = './references/nhl_rink.png'
with open(data_path) as json_file:
    data = json.load(json_file)

In [3]:
def get_game_ids(path):
    """returns a list of all game ids inside a json file"""
    with open(path) as json_file:
        data = json.load(json_file)
    game_ids = list(data.keys())
    return game_ids

game_ids = get_game_ids(data_path)

In [4]:
events_l = 100
vis_df = None
events = None

In [5]:
game_slider = IntSlider(
    value=min(game_ids),
    min=min(game_ids),
    max=max(game_ids),
    step=1,
    description='Game:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

    
event = IntSlider(
    value=0,
    min=0,
    max=events_l-1,
    step=1,
    description='Event: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')


display(game_slider, event)

out = Output()
display(out)


@out.capture()
def callback(game_id):
    global vis_df
    vis_df = data[str(game_id['new'])]
    all_plays = pd.DataFrame(vis_df['liveData']['plays']['allPlays'])
    event_ids = pd.json_normalize(all_plays['about'])['eventId'].values
    event.min = min(event_ids)
    event.max = max(event_ids)
    
    
def plot_image(coordinates, title):
    image = plt.imread("./references/nhl_rink.png")
    fig, ax = plt.subplots(figsize=(9,4))
    ax.set_xlabel('Feet', fontweight ='bold')
    ax.set_ylabel('Feet', fontweight ='bold')
    ax.set_title(title, fontsize = 10, fontweight ='bold')
    ax.imshow(image, extent=[-100, 100, 42.5, -42.5])
    plt.plot(coordinates['0_x'],coordinates['0_y'], "ob", markersize=8)
    plt.show()
    
    
@out.capture()
def plot_event(e):
    out.clear_output()
    
    a = "Game Start Date: " + vis_df['gameData']['datetime']['dateTime']
    display(a)
    b = vis_df['gameData']['teams']['home']['abbreviation']+" (home) vs "+vis_df['gameData']['teams']['away']['abbreviation']+" (away)"
    display(b)
    
    df = pd.DataFrame({'Team': [vis_df['gameData']['teams']['home']['abbreviation'], vis_df['gameData']['teams']['away']['abbreviation']],
                        'Goals': [vis_df['liveData']['linescore']['teams']['home']['goals'], vis_df['liveData']['linescore']['teams']['away']['goals']],
                        'SOG': [vis_df['liveData']['linescore']['teams']['home']['shotsOnGoal'], vis_df['liveData']['linescore']['teams']['away']['shotsOnGoal']],
                        'SO Goals': [vis_df['liveData']['linescore']['teams']['home']['powerPlay'], vis_df['liveData']['linescore']['teams']['away']['powerPlay']]                    
                        })  
    display(df)
    
    current_event_id = int(e['new'])

    all_plays = pd.DataFrame(vis_df['liveData']['plays']['allPlays'])
    about = pd.json_normalize(all_plays['about'])
    specific_event_about = {"about": about.loc[about['eventId'] == current_event_id].reset_index(drop=True).to_dict()}

    event_ids = pd.json_normalize(all_plays['about'])['eventId'].values
    coordinates = all_plays.coordinates.values
    players = all_plays.players.values
    team = all_plays.team.values
    result = all_plays.result.values

    con_data = pd.DataFrame(list(zip(event_ids,coordinates,players,team,result)), columns=['event_ids','coordinates','players','team','result'])
    
    specific_event = con_data.loc[about['eventId'] == current_event_id].reset_index(drop=True).to_dict()
    final_dict = {**specific_event_about, **specific_event}

    current_coordinates = pd.json_normalize(specific_event["coordinates"], sep='_').to_dict(orient='records')[0]
    plot_desc = pd.json_normalize(specific_event["result"], sep='_').to_dict(orient='records')[0]
    display(plot_image(current_coordinates, plot_desc["0_description"]))
    display(final_dict)
    plt.clf()


game_slider.observe(callback, names=['value'])
event.observe(plot_event, names=['value'])

IntSlider(value=2016030111, continuous_update=False, description='Game:', max=2016030417, min=2016030111)

IntSlider(value=0, continuous_update=False, description='Event: ', max=99)

Output()

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>